In [1]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import seaborn as sns
import re
import random as rd
import tensorflow as tf
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization
from nltk import word_tokenize, RegexpTokenizer
from nltk.corpus import stopwords, wordnet
from sklearn.preprocessing import OneHotEncoder
import matplotlib.pyplot as plt
import autokeras as ak
from sklearn.model_selection import KFold

import nltk
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ddayv\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\ddayv\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [2]:
def Tokenize(f):     ## Pre-processando a frase

    ## Colocando em minusculo
    ## Retirando a pontuaçao
    ## Retirando as StopWords
    
    f = f.lower().replace('\n', '').replace('-','').replace('#','').replace('.','').replace(',','').replace('!','').replace('r\n','').replace('  ','').replace('https','').replace('rt','').replace('rn','')
    token = RegexpTokenizer(r"\w+")
    f = token.tokenize(f)
    
    stop_words = set(stopwords.words('portuguese'))
    
    new_word = [word for word in f if not word in stop_words]
    
    return ' '.join(new_word)

def remove_user(frase):
    frase = re.sub('@\w+','',frase)
    frase = re.sub('{https}[^ ]+','',frase)
    frase = re.sub('https\w+','',frase)
    # re.sub('#\w+','',frase)
    return frase

def pre_X(frases):
    lista = []
    
    for frase in frases:
        lista.append(frase)
        
    return lista

def pre_Y(number):
    lista = []
    
    for numb in number:
        lista.append(numb)
    
    return lista

def set_array(frases):
    
    vocab = []
    palavras = []
    for frase in frases:
        
        text_array = remove_user(frase)
        text_array = Tokenize(text_array)
        text_array = text_array.split(' ')
        for i in range(len(text_array)):
            vocab.append(text_array[i])
    
    
        
    return vocab


## TWEETSENT

In [84]:
df_tweet= pd.read_csv('export_TweetSentBR.csv')
df_tweet

,id,id_twitter,text,sentiment
0,1343,863044774588272640,Que coisa linda! O Programa #encontro estava m...,1
1,1344,865583716088766467,"Por mais #Encontro com as Irmãs Galvão, adorei...",1
2,1345,865063232201011201,Mr. CATRA @OficialMrCatra lançando sua nova mú...,1
3,1346,864668391008763905,quem viu aquela lutadora modela barbuda tatuad...,0
4,1347,865572794016378882,Tô passada com esse cara.... quanta merda pode...,-1
...,...,...,...,...
11565,12908,864636619000877056,eu ja to aqui pronto pro #MasterChefBR mas ain...,-1
11566,12909,863581588713603072,MALUCO! Uma coisa que eu não tenho coragem é e...,-1
11567,12910,864831041349054464,#MaisVoce @ANAMARIABRAGA está linda @RedeGlobo,1
11568,12911,863042798575951872,"Que orgulho de ti, @sportrecife! #Encontro",1


In [85]:
df_remove = df_tweet[df_tweet['sentiment'] == '-']
df_tweet = df_tweet.drop(df_remove.index)

df_tweet['text'] = df_tweet['text'].apply(remove_user)
df_tweet['text'] = df_tweet['text'].apply(Tokenize)

In [86]:
df_tweet['sentiment'] = df_tweet['sentiment'].apply(lambda x: int(x))

In [87]:
text_Tweetsent = df_tweet['text']
polarity = np.asarray(df_tweet['sentiment'])

In [88]:
X_text = count_vect.transform(text_Tweetsent)

X_text_transform = tfidf_transformer.transform(X_text) # Aplicando o TF-IDF
X_text_transform.shape

(11533, 25852)

In [89]:
X_train1, X_test_TWEET, Y_train1, Y_test_TWEET = train_test_split(X_text_transform, polarity, test_size=0.3)

In [90]:
X_train12, X_test_TWEET1, Y_train12, Y_test_TWEET1 = train_test_split(df_tweet['text'], df_tweet['sentiment'] , test_size=0.3)

## TASH

In [91]:
df1 = pd.read_csv("tash-pt.csv")
df1 = df1.dropna()
df1 = df1.drop(columns=['id_twitter'])


In [92]:
df1['text'] = df1['text'].apply(remove_user)
df1['text'] = df1['text'].apply(Tokenize)

In [93]:
text_TASH = df1['text']
sentiment = np.asarray(df1['sentiment'])

X_text_TASH = count_vect.transform(text_TASH)
X_text_TASH_ = tfidf_transformer.transform(X_text_TASH) # Aplicando o TF-IDF
X_text_TASH_.shape

(2787, 25852)

In [94]:
X_train2, X_test_TASH, Y_train2, Y_test_TASH = train_test_split(X_text_TASH_, sentiment, test_size=0.3)

In [95]:
X_train21, X_test_TASH21, Y_train21, Y_test_TASH21 = train_test_split(df1['text'], np.asarray(df1['sentiment']), test_size=0.3)

## KANSOAN

In [96]:
df = pd.read_csv("data.txt", sep=',', header=None, names=['text','sentiment'])

df_remove = df[df['sentiment'] == '#Inveja'].index
df = df.drop(df_remove)

df_remove = df[df['sentiment'] == '#Raiva'].index
df = df.drop(df_remove)

df_remove = df[df['sentiment'] == '#Ironia'].index
df = df.drop(df_remove)


def binario(termo):
    if termo == '#Feliz' or termo == '#Amor':
        return 1
    elif termo == '#Triste' or termo == '#Chateado':
        return 0       
    
df['sentiment'] = df['sentiment'].apply(binario)

In [97]:
df['text'] = df['text'].apply(remove_user)
df['text'] = df['text'].apply(Tokenize)

In [98]:
text_KANSOAN = df['text']
sentiment = np.asarray(df['sentiment'])

X_text_KANSOAN = count_vect.transform(text_KANSOAN)
X_text_KANSOAN_ = tfidf_transformer.transform(X_text_KANSOAN) # Aplicando o TF-IDF
X_text_KANSOAN_.shape

(9224, 25852)

In [99]:
X_train3, X_test_KANSOAN, Y_train3, Y_test_KANSOAN = train_test_split(X_text_KANSOAN_, sentiment, test_size=0.3)

In [100]:
X_train31, X_test_KANSOAN1, Y_train31, Y_test_KANSOAN1 = train_test_split(df['text'], np.asarray(df['sentiment']), test_size=0.3)

## TEST

In [76]:
df = pd.read_excel('./rotulação/TweetsPolitical01 OK OK.xlsx')
df2 = pd.read_excel('./rotulação/TweetsPolitical02 OK OK.xlsx')
df3 = pd.read_excel('./rotulação/TweetsPolitical03 OK OK.xlsx')
df4 = pd.read_excel('./rotulação/TweetsPolitical04 OK OK.xlsx')
df5 = pd.read_excel('./rotulação/TweetsPolitical05 OK OK.xlsx')
df6 = pd.read_excel('./rotulação/TweetsPolitical06 OK OK.xlsx')
df7 = pd.read_excel('./rotulação/TweetsPolitical07 OK OK.xlsx')
df8 = pd.read_excel('./rotulação/TweetsPolitical08 OK OK.xlsx')
df9 = pd.read_excel('./rotulação/TweetsPolitical09 OK OK.xlsx')
df10 = pd.read_excel('./rotulação/TweetsPolitical10 OK OK.xlsx')
df11 = pd.read_excel('./rotulação/TweetsPolitical11 OK OK.xlsx')
df12 = pd.read_excel('./rotulação/TweetsPolitical12 OK OK.xlsx')
df13 = pd.read_excel('./rotulação/TweetsPolitical13 OK OK.xlsx')
df14 = pd.read_excel('./rotulação/TweetsPolitical14 OK OK.xlsx')
df15 = pd.read_excel('./rotulação/TweetsPolitical15 OK OK.xlsx')

lista = [df,df2,df3,df4,df5,df6,df7,df8,df9,df10,df11,df12,df13,df14,df15]

df_all = pd.concat(lista)
df_all.shape

(12670, 2)

In [77]:
df_all = df_all.dropna()

In [78]:
df_all.drop(df_all[df_all['Polaridade'] == 11].index, inplace=True)
df_all.drop(df_all[df_all['Polaridade'] == -2].index, inplace=True)
df_all.drop(df_all[df_all['Polaridade'] == 10].index, inplace=True)

df_all['Polaridade'].unique()

array([ 0.,  1., -1.])

In [79]:
df_all.head()

,Tweet,Polaridade
0,- #CaoNossoDeCadaDia #Novo - Vanessa Mandotti ...,0.0
1,"- Bola pra frente que amanhã é outro dia, outr...",1.0
2,- Cara de mal? Acho que não... Apenas um corte...,1.0
3,""" #PCdoB O PARTIDO QUE FAZ A DIFERENÇA: LEAL E...",1.0
4,"""@camaradamae: #PCdoB o Partido da coragem ! ...",1.0


In [80]:
df_all['Tweet'] = df_all['Tweet'].apply(remove_user)
df_all['Tweet'] = df_all['Tweet'].apply(Tokenize)
polarity = np.asarray(df_all['Polaridade'])
Tweet = df_all['Tweet']


In [81]:
count_vect = CountVectorizer()

Tweet = count_vect.fit_transform(Tweet)

tfidf_transformer = TfidfTransformer()
X_train_transform = tfidf_transformer.fit_transform(Tweet) # Aplicando o TF-IDF
X_train_transform.shape

(12626, 25852)

In [82]:
X_train, X_test, Y_train, Y_test = train_test_split(X_train_transform, polarity, test_size=0.3)



In [83]:
X_train_text, X_test_text, Y_train_text, Y_test_text = train_test_split(df_all['Tweet'] , df_all['Polaridade'], test_size=0.3)


## NB - TRAIN -> TASH- TEST -> UNILEX

In [101]:
clf = MultinomialNB()
clf.fit(X_train2, Y_train2)
y_pred = clf.predict(X_test)

print(classification_report(Y_test,y_pred))

              precision    recall  f1-score   support

        -1.0       0.46      0.22      0.30      1261
         0.0       0.41      0.77      0.54      1436
         1.0       0.47      0.20      0.28      1091

    accuracy                           0.43      3788
   macro avg       0.45      0.40      0.37      3788
weighted avg       0.44      0.43      0.39      3788



## NB - TRAIN -> TASH - TEST -> TWEET

In [102]:
clf = MultinomialNB()
clf.fit(X_train2, Y_train2)
y_pred = clf.predict(X_test_TASH)

print(classification_report(Y_test_TASH,y_pred))

              precision    recall  f1-score   support

          -1       0.47      0.31      0.38       267
           0       0.39      0.61      0.48       300
           1       0.44      0.30      0.36       270

    accuracy                           0.42       837
   macro avg       0.43      0.41      0.40       837
weighted avg       0.43      0.42      0.41       837



## NB - TRAIN -> TASH - TEST -> KANSOAN

In [103]:
def func(num):
    if num == 0:
        return -1
    else:
        return 1

sentiment = df['sentiment'].apply(func)
X_train3, X_test_KANSOAN, Y_train3, Y_test_KANSOAN = train_test_split(X_text_KANSOAN_, sentiment, test_size=0.3)

In [104]:
clf = MultinomialNB()
clf.fit(X_train2, Y_train2)
y_pred = clf.predict(X_test_KANSOAN)

print(classification_report(Y_test_KANSOAN,y_pred, zero_division=True))

              precision    recall  f1-score   support

          -1       0.94      0.56      0.70      1371
           0       0.00      1.00      0.00         0
           1       0.84      0.73      0.78      1397

    accuracy                           0.65      2768
   macro avg       0.59      0.76      0.50      2768
weighted avg       0.89      0.65      0.74      2768



## SVM - TRAIN->TASH - TEST->UNILEX

In [105]:
clf = SVC().fit(X_train2, Y_train2) 
y_pred = clf.predict(X_test)

print(classification_report(Y_test,y_pred))

              precision    recall  f1-score   support

        -1.0       0.49      0.06      0.11      1261
         0.0       0.39      0.96      0.56      1436
         1.0       0.68      0.07      0.13      1091

    accuracy                           0.40      3788
   macro avg       0.52      0.36      0.27      3788
weighted avg       0.51      0.40      0.28      3788



## SVM T TASH T TWEET

In [106]:
clf = SVC().fit(X_train2, Y_train2) 
y_pred = clf.predict(X_test_TWEET)

print(classification_report(Y_test_TWEET,y_pred))

              precision    recall  f1-score   support

          -1       0.50      0.23      0.32      1039
           0       0.28      0.77      0.41       847
           1       0.69      0.27      0.39      1574

    accuracy                           0.38      3460
   macro avg       0.49      0.42      0.37      3460
weighted avg       0.53      0.38      0.37      3460



## SVM T TASH T KANSOAN

In [107]:
clf = SVC().fit(X_train2, Y_train2) 
y_pred = clf.predict(X_test_KANSOAN)

print(classification_report(Y_test_KANSOAN,y_pred, zero_division=True))

              precision    recall  f1-score   support

          -1       0.95      0.50      0.65      1371
           0       0.00      1.00      0.00         0
           1       0.89      0.70      0.78      1397

    accuracy                           0.60      2768
   macro avg       0.61      0.73      0.48      2768
weighted avg       0.92      0.60      0.72      2768



## RL T TASH T UNILEX

In [108]:
clf = LogisticRegression(max_iter=1000).fit(X_train2, Y_train2) 
y_pred = clf.predict(X_test)

print(classification_report(Y_test,y_pred))

              precision    recall  f1-score   support

        -1.0       0.47      0.17      0.25      1261
         0.0       0.41      0.83      0.55      1436
         1.0       0.48      0.18      0.26      1091

    accuracy                           0.43      3788
   macro avg       0.45      0.39      0.35      3788
weighted avg       0.45      0.43      0.37      3788



## RL T TASH T TWEET

In [109]:
clf = LogisticRegression(max_iter=1000).fit(X_train2, Y_train2) 
y_pred = clf.predict(X_test_TWEET)

print(classification_report(Y_test_TWEET,y_pred))

              precision    recall  f1-score   support

          -1       0.49      0.33      0.39      1039
           0       0.29      0.60      0.39       847
           1       0.62      0.40      0.49      1574

    accuracy                           0.43      3460
   macro avg       0.47      0.44      0.42      3460
weighted avg       0.50      0.43      0.43      3460



## RL T TASH T KANSOAN

In [110]:
clf = LogisticRegression(max_iter=1000).fit(X_train2, Y_train2) 
y_pred = clf.predict(X_test_KANSOAN)

print(classification_report(Y_test_KANSOAN,y_pred, zero_division=True))

              precision    recall  f1-score   support

          -1       0.93      0.62      0.74      1371
           0       0.00      1.00      0.00         0
           1       0.87      0.77      0.82      1397

    accuracy                           0.70      2768
   macro avg       0.60      0.80      0.52      2768
weighted avg       0.90      0.70      0.78      2768



## TOTALMENT.C T TASH T UNILEX

In [111]:
model = tf.keras.Sequential([
    tf.keras.layers.Dense(50, activation='relu'),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Dense(25, activation='relu'),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Dense(10, activation='tanh'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(3 , activation='softmax')
])

model.compile(
    optimizer='adam',
    loss=tf.keras.losses.categorical_crossentropy,
    metrics=['accuracy']
)

one = OneHotEncoder(sparse=False)

y_one = one.fit_transform(np.asarray(Y_train2).reshape(-1,1))
y_one_test = one.fit_transform(Y_test.reshape(-1,1))

fit = model.fit(X_train2.todense(), y_one, epochs=10, validation_data=(X_test.todense(), y_one_test))

Epoch 1/10
61/61 [==============================] - 1s 22ms/step - loss: 1.0977 - accuracy: 0.3667 - val_loss: 1.0945 - val_accuracy: 0.3791
Epoch 2/10
61/61 [==============================] - 1s 12ms/step - loss: 1.0809 - accuracy: 0.3985 - val_loss: 1.0902 - val_accuracy: 0.3836
Epoch 3/10
61/61 [==============================] - 1s 12ms/step - loss: 0.9778 - accuracy: 0.5754 - val_loss: 1.0688 - val_accuracy: 0.4385
Epoch 4/10
61/61 [==============================] - 1s 13ms/step - loss: 0.6892 - accuracy: 0.7626 - val_loss: 1.1270 - val_accuracy: 0.4287
Epoch 5/10
61/61 [==============================] - 1s 14ms/step - loss: 0.4219 - accuracy: 0.8708 - val_loss: 1.2477 - val_accuracy: 0.4221
Epoch 6/10
61/61 [==============================] - 1s 12ms/step - loss: 0.2743 - accuracy: 0.9277 - val_loss: 1.4429 - val_accuracy: 0.4263
Epoch 7/10
61/61 [==============================] - 1s 13ms/step - loss: 0.1750 - accuracy: 0.9554 - val_loss: 1.6147 - val_accuracy: 0.4256
Epoch 8/10
61

In [112]:
predicted = model.predict(X_test.todense())

print(classification_report(np.argmax(y_one_test, axis=1), np.argmax(predicted, axis=1)))

              precision    recall  f1-score   support

           0       0.40      0.41      0.40      1261
           1       0.43      0.56      0.49      1436
           2       0.41      0.24      0.30      1091

    accuracy                           0.42      3788
   macro avg       0.41      0.40      0.40      3788
weighted avg       0.41      0.42      0.41      3788



## TWEET

In [113]:
predicted = model.predict(X_test_TWEET.todense())

print(classification_report(np.argmax(one.fit_transform(Y_test_TWEET.reshape(-1,1)), axis=1), np.argmax(predicted, axis=1)))

              precision    recall  f1-score   support

           0       0.42      0.33      0.37      1039
           1       0.28      0.49      0.36       847
           2       0.57      0.43      0.49      1574

    accuracy                           0.41      3460
   macro avg       0.43      0.42      0.41      3460
weighted avg       0.46      0.41      0.42      3460



## KANSOAN

In [114]:
predicted = model.predict(X_test_KANSOAN.todense())

print(classification_report(np.argmax(one.fit_transform(np.asarray(Y_test_KANSOAN).reshape(-1,1)), axis=1), np.argmax(predicted, axis=1)))

              precision    recall  f1-score   support

           0       0.86      0.69      0.76      1371
           1       0.64      0.34      0.44      1397
           2       0.00      0.00      0.00         0

    accuracy                           0.51      2768
   macro avg       0.50      0.34      0.40      2768
weighted avg       0.75      0.51      0.60      2768



c:\users\ddayv\appdata\local\programs\python\python37\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## LSTM T TASH T TWEET

In [115]:
vectorize_layer = TextVectorization(
 max_tokens=15000,
 output_mode='int',
 output_sequence_length=len(max(df1['text'])))

vocab = set_array(df1['text'])

vectorize_layer.adapt(np.unique(vocab))
len(vectorize_layer.get_vocabulary())


11562

In [116]:
text_KANSOAN

X_train3_, X_test_KANSOAN_, Y_train3_, Y_test_KANSOAN_ = train_test_split(text_KANSOAN, df['sentiment'], test_size=0.3)

In [117]:
model = tf.keras.Sequential([
    vectorize_layer,
    tf.keras.layers.Embedding(
        input_dim=len(vectorize_layer.get_vocabulary()),
        output_dim=64,mask_zero=True),
    
    tf.keras.layers.LSTM(50, activation='relu' ,return_sequences=True),
    tf.keras.layers.Dropout(0.3),
    
    tf.keras.layers.LSTM(25 , activation='tanh', return_sequences=True),
    tf.keras.layers.Dropout(0.3),
    
    tf.keras.layers.LSTM(10 , activation='tanh'),
    tf.keras.layers.Dropout(0.2),
    
    tf.keras.layers.Dense(3, activation='softmax')
])


model.compile(
    optimizer= tf.keras.optimizers.Adam(),
    loss=tf.keras.losses.categorical_crossentropy,
    metrics=['accuracy']
)

y_one = one.fit_transform(np.asarray(Y_train21).reshape(-1,1))
y_one_test = one.fit_transform(np.asarray(Y_test_TWEET1).reshape(-1,1))

fit = model.fit(np.asarray(pre_X(X_train21)), y_one, epochs=10, batch_size=128 ,validation_data=(np.asarray(pre_X(X_test_TWEET1)), y_one_test))

Epoch 1/10
16/16 [==============================] - 4s 238ms/step - loss: 1.0982 - accuracy: 0.3369 - val_loss: 1.1017 - val_accuracy: 0.2624
Epoch 2/10
16/16 [==============================] - 2s 108ms/step - loss: 1.0938 - accuracy: 0.3492 - val_loss: 1.1038 - val_accuracy: 0.2624
Epoch 3/10
16/16 [==============================] - 2s 116ms/step - loss: 1.0762 - accuracy: 0.4241 - val_loss: 1.1041 - val_accuracy: 0.2890
Epoch 4/10
16/16 [==============================] - 2s 122ms/step - loss: 0.9055 - accuracy: 0.5995 - val_loss: 1.1447 - val_accuracy: 0.3416
Epoch 5/10
16/16 [==============================] - 2s 117ms/step - loss: 0.6549 - accuracy: 0.7467 - val_loss: 1.2263 - val_accuracy: 0.3471
Epoch 6/10
16/16 [==============================] - 2s 124ms/step - loss: 0.4747 - accuracy: 0.8754 - val_loss: 1.3840 - val_accuracy: 0.3434
Epoch 7/10
16/16 [==============================] - 2s 116ms/step - loss: 0.3860 - accuracy: 0.9082 - val_loss: 1.4357 - val_accuracy: 0.3286
Epoch 

In [118]:
predicted = model.predict(np.asarray(np.asarray(pre_X(X_test_TWEET1))))

print(classification_report(np.argmax(one.fit_transform(np.asarray(Y_test_TWEET1).reshape(-1,1)), axis=1), np.argmax(predicted, axis=1)))

              precision    recall  f1-score   support

           0       0.29      0.45      0.35      1009
           1       0.25      0.34      0.29       908
           2       0.56      0.23      0.33      1543

    accuracy                           0.33      3460
   macro avg       0.37      0.34      0.32      3460
weighted avg       0.40      0.33      0.33      3460



## UNILEX

In [119]:
predicted = model.predict(np.asarray(pre_X(X_test_text)))

print(classification_report(np.argmax(one.fit_transform(np.asarray(Y_test_text).reshape(-1,1)), axis=1), np.argmax(predicted, axis=1)))

              precision    recall  f1-score   support

           0       0.36      0.64      0.46      1246
           1       0.38      0.34      0.36      1443
           2       0.47      0.12      0.19      1099

    accuracy                           0.37      3788
   macro avg       0.40      0.37      0.34      3788
weighted avg       0.40      0.37      0.34      3788



## KANSOAN

In [120]:
predicted = model.predict(np.asarray(pre_X(X_test_KANSOAN1)))

print(classification_report(np.argmax(one.fit_transform(np.asarray(Y_test_KANSOAN1).reshape(-1,1)), axis=1), np.argmax(predicted, axis=1)))

              precision    recall  f1-score   support

           0       0.60      0.58      0.59      1360
           1       0.53      0.29      0.38      1408
           2       0.00      0.00      0.00         0

    accuracy                           0.43      2768
   macro avg       0.38      0.29      0.32      2768
weighted avg       0.57      0.43      0.48      2768



c:\users\ddayv\appdata\local\programs\python\python37\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## CONV1D T TASH T KANSOAN

In [121]:
model = tf.keras.Sequential([
    vectorize_layer,
    tf.keras.layers.Embedding(
        input_dim=len(vectorize_layer.get_vocabulary()),
        output_dim=64,
        mask_zero=True),
    
    tf.keras.layers.Conv1D(32,6, activation='relu'),
    tf.keras.layers.MaxPooling1D(2),
    tf.keras.layers.Flatten(),
    
    tf.keras.layers.Dense(16, activation='relu'),
    tf.keras.layers.Dense(3, activation='softmax')   
])

model.compile(
    optimizer= tf.keras.optimizers.Adam(),
    loss=tf.keras.losses.categorical_crossentropy,
    metrics=['accuracy']
)

y_one = one.fit_transform(np.asarray(Y_train21).reshape(-1,1))
y_one_test = one.fit_transform(Y_test_KANSOAN1.reshape(-1,1))

fit = model.fit(np.asarray(pre_X(X_train21)), y_one, epochs=10, batch_size=128)

Epoch 1/10
16/16 [==============================] - 0s 15ms/step - loss: 1.0987 - accuracy: 0.3472
Epoch 2/10
16/16 [==============================] - 0s 16ms/step - loss: 1.0830 - accuracy: 0.3754
Epoch 3/10
16/16 [==============================] - 0s 14ms/step - loss: 1.0395 - accuracy: 0.4662
Epoch 4/10
16/16 [==============================] - 0s 14ms/step - loss: 0.9234 - accuracy: 0.7990
Epoch 5/10
16/16 [==============================] - 0s 15ms/step - loss: 0.6834 - accuracy: 0.9251
Epoch 6/10
16/16 [==============================] - 0s 16ms/step - loss: 0.3798 - accuracy: 0.9662
Epoch 7/10
16/16 [==============================] - 0s 15ms/step - loss: 0.1785 - accuracy: 0.9810
Epoch 8/10
16/16 [==============================] - 0s 15ms/step - loss: 0.0877 - accuracy: 0.9908
Epoch 9/10
16/16 [==============================] - 0s 16ms/step - loss: 0.0498 - accuracy: 0.9949
Epoch 10/10
16/16 [==============================] - 0s 15ms/step - loss: 0.0298 - accuracy: 0.9985


In [122]:
predicted = model.predict(np.asarray(pre_X(X_test_KANSOAN1)))

print(classification_report(np.argmax(one.fit_transform(np.asarray(Y_test_KANSOAN1).reshape(-1,1)), axis=1), np.argmax(predicted, axis=1)))

              precision    recall  f1-score   support

           0       0.69      0.79      0.74      1360
           1       0.43      0.07      0.12      1408
           2       0.00      0.00      0.00         0

    accuracy                           0.42      2768
   macro avg       0.37      0.28      0.28      2768
weighted avg       0.56      0.42      0.42      2768



c:\users\ddayv\appdata\local\programs\python\python37\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## UNILEX

In [123]:
predicted = model.predict(np.asarray(pre_X(X_test_text)))

print(classification_report(np.argmax(one.fit_transform(np.asarray(Y_test_text).reshape(-1,1)), axis=1), np.argmax(predicted, axis=1)))

              precision    recall  f1-score   support

           0       0.34      0.78      0.47      1246
           1       0.34      0.09      0.14      1443
           2       0.40      0.19      0.26      1099

    accuracy                           0.35      3788
   macro avg       0.36      0.35      0.29      3788
weighted avg       0.36      0.35      0.28      3788



## TWEET

In [124]:
predicted = model.predict(np.asarray(pre_X(X_test_TWEET1)))

print(classification_report(np.argmax(one.fit_transform(np.asarray(Y_test_TWEET1).reshape(-1,1)), axis=1), np.argmax(predicted, axis=1)))

              precision    recall  f1-score   support

           0       0.28      0.56      0.38      1009
           1       0.25      0.11      0.16       908
           2       0.51      0.34      0.41      1543

    accuracy                           0.35      3460
   macro avg       0.35      0.34      0.32      3460
weighted avg       0.38      0.35      0.34      3460



## BDR T TASH T UNILEX

In [125]:
model = tf.keras.Sequential([
    vectorize_layer,
    tf.keras.layers.Embedding(
        input_dim=len(vectorize_layer.get_vocabulary()),
        output_dim=1,
        mask_zero=True),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(1)),

    tf.keras.layers.Dense(3, activation='softmax')
])

model.compile(
    optimizer= tf.keras.optimizers.Adam(),
    loss=tf.keras.losses.categorical_crossentropy,
    metrics=['accuracy']
)

y_one = one.fit_transform(np.asarray(Y_train21).reshape(-1,1))
y_one_test = one.fit_transform(np.asarray(Y_test_text).reshape(-1,1))

fit = model.fit(np.asarray(pre_X(X_train21)), y_one, epochs=10, batch_size=128 ,validation_data=(np.asarray(pre_X(X_test_text)), y_one_test))

Epoch 1/10
16/16 [==============================] - 2s 153ms/step - loss: 1.0984 - accuracy: 0.3462 - val_loss: 1.0984 - val_accuracy: 0.3807
Epoch 2/10
16/16 [==============================] - 0s 24ms/step - loss: 1.0959 - accuracy: 0.3651 - val_loss: 1.0979 - val_accuracy: 0.3820
Epoch 3/10
16/16 [==============================] - 0s 27ms/step - loss: 1.0936 - accuracy: 0.3585 - val_loss: 1.0975 - val_accuracy: 0.3809
Epoch 4/10
16/16 [==============================] - 0s 28ms/step - loss: 1.0904 - accuracy: 0.3677 - val_loss: 1.0969 - val_accuracy: 0.3862
Epoch 5/10
16/16 [==============================] - 0s 28ms/step - loss: 1.0859 - accuracy: 0.4072 - val_loss: 1.0959 - val_accuracy: 0.3986
Epoch 6/10
16/16 [==============================] - 0s 30ms/step - loss: 1.0787 - accuracy: 0.5215 - val_loss: 1.0943 - val_accuracy: 0.4195
Epoch 7/10
16/16 [==============================] - 0s 27ms/step - loss: 1.0675 - accuracy: 0.5913 - val_loss: 1.0922 - val_accuracy: 0.4155
Epoch 8/10
1

In [126]:
predicted = model.predict(np.asarray(pre_X(X_test_text)))

print(classification_report(np.argmax(one.fit_transform(np.asarray(Y_test_text).reshape(-1,1)), axis=1), np.argmax(predicted, axis=1)))

              precision    recall  f1-score   support

           0       0.45      0.58      0.51      1246
           1       0.44      0.67      0.53      1443
           2       0.00      0.00      0.00      1099

    accuracy                           0.45      3788
   macro avg       0.30      0.42      0.35      3788
weighted avg       0.32      0.45      0.37      3788



c:\users\ddayv\appdata\local\programs\python\python37\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## TWEET

In [127]:
predicted = model.predict(np.asarray(pre_X(X_test_TWEET1)))

print(classification_report(np.argmax(one.fit_transform(np.asarray(Y_test_TWEET1).reshape(-1,1)), axis=1), np.argmax(predicted, axis=1)))

              precision    recall  f1-score   support

           0       0.41      0.28      0.33      1009
           1       0.26      0.80      0.40       908
           2       0.00      0.00      0.00      1543

    accuracy                           0.29      3460
   macro avg       0.22      0.36      0.24      3460
weighted avg       0.19      0.29      0.20      3460



c:\users\ddayv\appdata\local\programs\python\python37\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## KANSOAN

In [128]:
predicted = model.predict(np.asarray(pre_X(X_test_KANSOAN1)))

print(classification_report(np.argmax(one.fit_transform(np.asarray(Y_test_KANSOAN1).reshape(-1,1)), axis=1), np.argmax(predicted, axis=1)))

              precision    recall  f1-score   support

           0       0.79      0.68      0.73      1360
           1       0.73      0.82      0.77      1408

    accuracy                           0.75      2768
   macro avg       0.76      0.75      0.75      2768
weighted avg       0.76      0.75      0.75      2768

